In [3]:
#from rltensor.app import APPAgent
import pickle
import trader
from trader import BFXClient, Trader
from datetime import datetime
from time import sleep
import importlib

# <TEMP>
import pandas as pd
import numpy as np
import sqlite3
# </TEMP>

import sys
sys.path.append('/Users/zach-eberhart/desktop/rltrader/pytrade_env/pytrade_env/database')
import trade_history
from trade_history import TradeHistory

# dep: pandas 0.22.0
sys.path.append('/Users/zach-eberhart/desktop/rltrader/rltensor/execution/personal_online')
import online_helpers as p
import twitter

In [4]:
def get_weights_dict(weights, currency_pairs):
    dict_ = {}
    for cp, w in zip(currency_pairs, weights):
        dict_[cp] = w
    return dict_

def get_price_dict(currency_pairs):
    k = 'XeF1yB3isJHwgDgbk2WS6wGjxZwEGT4mNbwFqAoCMyv'
    s = 'sD1jXrh4FCzNvKOPWEgYOhUTJFCnh0PPxbhbtVVPEmQ'
    bfx = BFXClient(k, s)
    return bfx.get_last_price(currency_pairs).to_dict()

def period_result(weights, currency_pairs):
    weights_dict = get_weights_dict(weights, currency_pairs)
    price_dict = get_price_dict(currency_pairs)
    current_dt = str(datetime.utcnow().strftime("%Y-%m-%d %H:%M:%S"))
    return {'datetime': current_dt, 'weights': weights_dict, 'price': price_dict}

def get_period_end():
    now = [int(i) for i in datetime.now().strftime('%Y %m %d %H %M').split()]
    if (int(now[4]) - 30) < 0:
        now[4] = 30
    else:
        now[3] += 1
        if now[3] == 24:
            now[2] += 1
            now[3] = 0
        now[4] = 0
    y, M, d, h, m = now
    period_end = int(datetime(y, M, d, h, m).timestamp())
    return period_end

def sleep_until_period_end(sleep_buffer = 5):
    period_end = get_period_end()
    now = int(datetime.now().timestamp())
    sleep_time = period_end - now + sleep_buffer
    print('Sleeping until period end (%s seconds).' % sleep_time)
    sleep(sleep_time)

# RLTrader

In [5]:
tickers = []

with (open("/Users/zach-eberhart/desktop/rltrader/rltensor/rltensor/app/ticker1.pkl", "rb")) as openfile:
    
    while True:
        try: tickers.append(pickle.load(openfile))
        except EOFError: break

currency_pairs = tickers[0]

print('\nCurrency Pairs Used in Test:\n')
print(currency_pairs)

alternative_portfolio = ['USDT_BCH', 'USDT_XMR', 'USDT_DASH',
                         'USDT_NEO', 'USDT_EOS', 'USDT_SAN', 
                         'USDT_IOTA', 'USDT_BTC', 'USDT_ETH']

print('\nCurrency Pairs Used in Live Online Algo:\n')
print(alternative_portfolio)

all_relevant_pairs = list(set(currency_pairs + alternative_portfolio))


Currency Pairs Used in Test:

['USDT_BCH', 'USDT_BTC', 'USDT_DASH', 'USDT_ETC', 'USDT_ETH', 'USDT_LTC', 'USDT_XMR', 'USDT_XRP', 'USDT_ZEC']

Currency Pairs Used in Live Online Algo:

['USDT_BCH', 'USDT_XMR', 'USDT_DASH', 'USDT_NEO', 'USDT_EOS', 'USDT_SAN', 'USDT_IOTA', 'USDT_BTC', 'USDT_ETH']


## Pre-Retrieve New Data (if needed)

In [7]:
p.just_update_trade_history(alternative_portfolio)


=== Getting Trade History ===

Period End: 1536424200
Currency Pair: USDT_BCH
Latest Trade: 1536393613000
API Start: 1536393613000
API End: 1536423702457
Successfully added trades to raw db.

Currency Pair: USDT_XMR
Latest Trade: 1536394279000
API Start: 1536394279000
API End: 1536423712120
Successfully added trades to raw db.

Currency Pair: USDT_DASH
Latest Trade: 1536394682000
API Start: 1536394682000
API End: 1536423715303
Successfully added trades to raw db.

Currency Pair: USDT_NEO
Latest Trade: 1536395166000
API Start: 1536395166000
API End: 1536423727990
Successfully added trades to raw db.

Currency Pair: USDT_EOS
Latest Trade: 1536396173000
API Start: 1536396173000
API End: 1536423740707
Successfully added trades to raw db.

Currency Pair: USDT_SAN
Latest Trade: 1536398793000
API Start: 1536398793000
API End: 1536423760094
Successfully added trades to raw db.

Currency Pair: USDT_IOTA
Latest Trade: 1536399287000
API Start: 1536399287000
API End: 1536423763230
Successfully ad

'success'

In [5]:
# df = history.get_processed_charts_from_db()[['open', 'high', 'low', 'close', 'volume', 'currency_pair']].copy()
# df[df.index == df.index.max()]

## ONS (Balance Only)

In [6]:
# VPN

# balance portfolio
accounts = ['l']#, 'p']

# get ONS weights
latest_weights = p.get_online_weights(alternative_portfolio)

# balance portfolio
for a in accounts:
    trade = trader.Trader(account = a)
    trade.rebalance_portfolio(asset_weights = latest_weights.iloc[-1,:].copy(),
                              fill_order_premium = 0.025,
                              order_minimum = 10)


=== Getting Trade History ===

Period End: 1531094400
Currency Pair: USDT_BCH
Latest Trade: 1531088068000
API Start: 1531088068000
API End: 1531092619472
Successfully added trades to raw db.

Currency Pair: USDT_XMR
Latest Trade: 1531088132000
API Start: 1531088132000
API End: 1531092641632
Successfully added trades to raw db.

Currency Pair: USDT_DASH
Latest Trade: 1531087347000
API Start: 1531087347000
API End: 1531092645692
Successfully added trades to raw db.

Currency Pair: USDT_NEO
Latest Trade: 1531088112000
API Start: 1531088112000
API End: 1531092649659
Successfully added trades to raw db.

Currency Pair: USDT_EOS
Latest Trade: 1531088152000
API Start: 1531088152000
API End: 1531092661254
Successfully added trades to raw db.

Currency Pair: USDT_SAN
Latest Trade: 1531087648000
API Start: 1531087648000
API End: 1531092670746
Successfully added trades to raw db.

Currency Pair: USDT_IOTA
Latest Trade: 1531088131000
API Start: 1531088131000
API End: 1531092674822
Successfully ad

KeyError: 'tADDUSD'

## Weights for Twitter

In [6]:
latest_weights = p.get_online_weights(alternative_portfolio)
tweet = twitter.get_tweet(latest_weights)


=== Getting Trade History ===

Period End: 1533400200
Currency Pair: USDT_BCH
Latest Trade: 1533397017000
API Start: 1533397017000
API End: 1533399269223
Successfully added trades to raw db.

Currency Pair: USDT_XMR
Latest Trade: 1533397041000
API Start: 1533397041000
API End: 1533399272427
Successfully added trades to raw db.

Currency Pair: USDT_DASH
Latest Trade: 1533397090000
API Start: 1533397090000
API End: 1533399275643
Successfully added trades to raw db.

Currency Pair: USDT_NEO
Latest Trade: 1533397071000
API Start: 1533397071000
API End: 1533399278853
Successfully added trades to raw db.

Currency Pair: USDT_EOS
Latest Trade: 1533397114000
API Start: 1533397114000
API End: 1533399282023
Successfully added trades to raw db.

Currency Pair: USDT_SAN
Latest Trade: 1533397197000
API Start: 1533397197000
API End: 1533399288551
Successfully added trades to raw db.

Currency Pair: USDT_IOTA
Latest Trade: 1533397224000
API Start: 1533397224000
API End: 1533399291682
Successfully ad

In [7]:
twitter.post_tweet(tweet)

successfully tweeted: Most Weight: $DASH (30.4%, -0.5%)
Least Weight: $EOS (0.0%, -15.3%)
Biggest Mover: $XMR (12.9%, 4.7%)

#crypto #cryptotrading


## ONS (Continuous Update)

In [ ]:
# results = []

# sleep_until_period_end()

# while True:
    
#     # update trade history
#     p.just_update_trade_history(all_relevant_pairs, verbose = False)
    
#     # if it's between 00:00-00:30 UTC, then rebalance portfolio
#     if int(datetime.now().strftime('%M')) < 30 and \
#        int(datetime.utcnow().strftime('%H')) == 0:
        
#         print('\n\n\n\n\n========== REBALANCING PORTFOLIO ==========\n\n\n')
            
#         # get ONS weights
#         latest_weights = p.get_online_weights(alternative_portfolio)
        
#         # balance portfolio
#         accounts = ['p', 'l']
#         for a in accounts:
#             trade = trader.Trader(account = a)
#             trade.rebalance_portfolio(asset_weights = latest_weights.iloc[-1,:].copy(),
#                                       fill_order_premium = 0.025,
#                                       order_minimum = 10)
        
#         # post to twitter
#         tweet = twitter.get_tweet(latest_weights)
#         twitter.post_tweet(tweet)
    
#     # sleep, update, sleep
#     sleep(1200)
#     p.just_update_trade_history(all_relevant_pairs, verbose = False)
#     print()
#     sleep_until_period_end()
#     print('\n\n')

## Train Model

In [6]:
str(datetime.utcnow().strftime("%Y-%m-%d %H:%M:%S"))

'2018-02-27 00:17:49'

In [7]:
start = '2017-08-03 00:00:00'
end = '2018-02-26 22:00:00'

load_file_path = "params2016-02-01_00:00:00/model.ckpt"
num_epochs = 1

In [9]:
APPAgent.fit(start, end, load_file_path, num_epochs)

# Live Testing

In [7]:
# del trade, trader
# import trader
# importlib.reload(trader)

In [9]:
results = []

#sleep_until_period_end()

while True:
    
    # update model and get prediction
    APPAgent.update_data()
    APPAgent.update_model(num_epochs = 1)
    pred = APPAgent.predict()
    
    # save results
    current_result = dict(period_result(pred, currency_pairs))
    results.append(current_result)
    print(current_result, '\n\n')
    
    '''
    TODO: SORT THE ASSETS TO BALANCE SO THAT NEGATIVES COME FIRST, AND THEN THE
    HIGHEST POSITIVES (SO WE GET THE MOST OF THE ASSET THAT IS WEIGHED THE MOST)
    '''
    
    # set account
    accounts = ['p', 'l']
    
    if int(datetime.now().strftime('%M')) < 30 and \
       int(datetime.now().strftime('%H')) % 2 == 0 and False:
        
        # balance portfolio
        for a in accounts:
            trade = trader.Trader(account = a)
            trade.rebalance_portfolio(asset_weights = pd.Series(current_result['weights']),
                                      fill_order_premium = 0.01,
                                      order_minimum = 10)
    
    
    if int(datetime.now().strftime('%M')) < 30 and \
       int(datetime.utcnow().strftime('%H')) == 0:
        
        print('\n\n\n\n\n========== REBALANCING PORTFOLIO ==========\n\n\n')
            
        # get ONS weights
        latest_weights = p.get_online_weights(alternative_portfolio)
        
        # balance portfolio
        for a in accounts:
            trade = trader.Trader(account = a)
            trade.rebalance_portfolio(asset_weights = latest_weights,
                                      fill_order_premium = 0.025,
                                      order_minimum = 10)
    
    
    # sleep for a little and then update
    sleep(1200)
    p.just_update_trade_history(all_relevant_pairs, verbose = False)
    
    # and now sleep until the end of the period
    print()
    sleep_until_period_end()
    print('\n\n')

## Recent Account Values

In [9]:
trader.Trader(account = 'p').get_account_value()

Account: p
Fetching exchange


KeyError: 'tADDUSD'

In [7]:
trader.Trader(account = 'l').get_account_value()

Account: l
Fetching margin
Fetching funding
Fetching exchange


KeyError: 'tADDUSD'

# Export for Reporting

In [8]:
export_date = '2018-09-01'

In [9]:
latest_weights = p.get_online_weights(alternative_portfolio)

history = TradeHistory(currency_pairs = alternative_portfolio,
                           time_period = '24h',
                           data_dir = '/Users/zach-eberhart/desktop/crypto/_hft/'
                      )

df = history.get_processed_charts_from_db()[['close', 'currency_pair']].copy()
df = history.reformat_for_online_algo(df, ['close'])

if df.shape == latest_weights.shape:
    
    # export
    weights_fname = 'portfolio_weights_export_%s.csv' % export_date
    close_fname = 'portfolio_close_export_%s.csv' % export_date
    latest_weights.index = df.index
    latest_weights.to_csv(weights_fname)
    df.to_csv(close_fname)
    
    print('successfully exported data')
    
else:
    print('dataframes are not the same shape')
    print('closing prices df shape:', df.shape)
    print('weights df shape:', latest_weights.shape)


=== Getting Trade History ===

Period End: 1536426000
Currency Pair: USDT_BCH
Latest Trade: 1536423575000
API Start: 1536423575000
API End: 1536425183214
Successfully added trades to raw db.

Currency Pair: USDT_XMR
Latest Trade: 1536423181000
API Start: 1536423181000
API End: 1536425186459
Successfully added trades to raw db.

Currency Pair: USDT_DASH
Latest Trade: 1536423713000
API Start: 1536423713000
API End: 1536425189607
Successfully added trades to raw db.

Currency Pair: USDT_NEO
Latest Trade: 1536423682000
API Start: 1536423682000
API End: 1536425192760
Successfully added trades to raw db.

Currency Pair: USDT_EOS
Latest Trade: 1536423729000
API Start: 1536423729000
API End: 1536425196002
Successfully added trades to raw db.

Currency Pair: USDT_SAN
Latest Trade: 1536423396000
API Start: 1536423396000
API End: 1536425199142
Successfully added trades to raw db.

Currency Pair: USDT_IOTA
Latest Trade: 1536423617000
API Start: 1536423617000
API End: 1536425202255
Successfully ad